In [ ]:

import os
import numpy as np
import cv2 as cv
import mediapipe as mp

# Constants
DELAY = 50
INPUT_TYPES = ['palm', 'fist', 'thumbsup', 'gun', 'call']
MIN_INPUT_COUNT = 2000 + DELAY

def make_dirs():
    global INPUT_TYPES
    directory = './Inputs/'
    if not os.path.isdir(directory):
        os.mkdir(directory, mode=511)
    os.chdir(directory)
    for dirs in INPUT_TYPES:
        if not os.path.isdir(dirs):
            os.mkdir(dirs, mode=511)
    os.chdir('..')

# Initialize Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

# Capture from webcam
webcam = 0
capture = cv.VideoCapture(webcam)
fps = int(capture.get(cv.CAP_PROP_FPS))
print("fps is " + str(fps))
_, frame = capture.read()
height, width, _ = frame.shape

# Loop through input types
for types in INPUT_TYPES:
    count = 0
    while capture.isOpened():
        if (cv.waitKey(1) & 0xFF == 13) or (count >= MIN_INPUT_COUNT + 1):
            break
        black = np.zeros(shape=frame.shape)
        _, frame = capture.read()
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        result = holistic.process(frame_rgb)
        try:
            hand_landmarks = result.right_hand_landmarks.landmark
            if hand_landmarks:
                x_max = 0
                y_max = 0
                x_min = width
                y_min = height
                for i in range(21):
                    lm = hand_landmarks[i]
                    x, y = int(lm.x * width), int(lm.y * height)
                    if x > x_max:
                        x_max = x
                    if x < x_min:
                        x_min = x
                    if y > y_max:
                        y_max = y
                    if y < y_min:
                        y_min = y
                frame_bgr = cv.cvtColor(frame_rgb, cv.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(frame_bgr, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                cv.rectangle(frame_bgr, (x_min - 25, y_min - 25), (x_max + 25, y_max + 25), (0, 255, 0), 2)
                result1 = frame_bgr
                mp_drawing.draw_landmarks(black, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                cropped = black[y_min - 23: y_max + 23, x_min - 23: x_max + 23]
                resized = cv.resize(cropped, (96, 96))
                result2 = cv.flip(resized, 1)
                cv.imshow("Frame2", result2)
        except:
            result1 = frame
            pass
        mirror1 = cv.flip(result1, 1)
        if count <= DELAY:
            delay_msg = "Class of " + types + " is going to start in " + str(DELAY - count)
            cv.putText(mirror1, delay_msg, (20, 20), cv.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1)
        else:
            msg = "Class of " + types + " input no : " + str(count - DELAY)
            cv.putText(mirror1, msg, (50, 50), cv.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 1)
        try:
            path_mirror2 = "Inputs/" + types + "/" + types + "_" + str(count - DELAY) + ".jpg"
            # cv.imwrite(path_mirror2, result2)
        except:
            pass
        count += 1
        cv.imshow('frame1', mirror1)
capture.release()
cv.destroyAllWindows()
